# Sudoku Solver

In [1]:
import os
import platform

- **os module:** Provides functions for interacting with the operating system, like executing commands (`os.system()`) and manipulating files (`os.listdir()`).
- **platform module:** Provides functions to retrieve platform-specific information such as the operating system name (`platform.system()`) and machine type (`platform.machine()`).

In [2]:
CLEAR_COMMAND = 'cls' if platform.system() == 'Windows' else 'clear'

def clear_screen():
    os.system(CLEAR_COMMAND)

- **CLEAR_COMMAND:** Sets `'cls'` for Windows or `'clear'` for Unix-like systems based on the platform detected by `platform.system()`.
- **clear_screen():** Executes the appropriate command (`'cls'` or `'clear'`) using `os.system()` to clear the console screen.

In [3]:
def load_puzzle_data(grid_size, percentage):
    filename = 'pattern.txt'
    target_label = f'{grid_size}x{grid_size} {percentage}% puzzles'

    with open(filename, 'r') as file:
        content = file.read()
        puzzles = content.split('\n\n')  # Each puzzle is separated by two newline characters

    for puzzle_section in puzzles:
        if target_label in puzzle_section:
            puzzle_lines = puzzle_section.split('\n')
            puzzle = ''.join(puzzle_lines[1])
            return puzzle

    print(f"No puzzle found for size {grid_size} and percentage {percentage}.")
    return ""

# Function to get user input for grid size or percentage
def get_user_input(prompt, valid_inputs):
    while True:
        try:
            user_input = int(input(prompt))
            if user_input in valid_inputs:
                return user_input
            else:
                print(f"Invalid input. Please enter one of: {', '.join(map(str, valid_inputs))}")
        except ValueError:
            print("Invalid input. Please enter a number.")

- **load_puzzle_data(grid_size, percentage):** Loads puzzle data from a file (`pattern.txt`) based on specified grid size and percentage, returning the puzzle as a string or an empty string if not found.
  
- **get_user_input(prompt, valid_inputs):** Prompts the user with a message (`prompt`) and validates input against a list of valid values (`valid_inputs`), ensuring the input is an integer and within the valid range.

In [4]:
def solve_sudoku(board, grid_size):
    def is_valid(board, num, pos):
        # Check row and column constraints
        for i in range(grid_size):
            if board[pos[0]][i] == num and pos[1] != i:
                return False
            if board[i][pos[1]] == num and pos[0] != i:
                return False

        # Check subgrid constraints
        subgrid_size = int(grid_size ** 0.5)
        subgrid_x = pos[1] // subgrid_size
        subgrid_y = pos[0] // subgrid_size

        for i in range(subgrid_y * subgrid_size, subgrid_y * subgrid_size + subgrid_size):
            for j in range(subgrid_x * subgrid_size, subgrid_x * subgrid_size + subgrid_size):
                if board[i][j] == num and (i, j) != pos:
                    return False

        return True

    # Find an empty cell in the Sudoku grid
    def find_empty_cell(board):
        for i in range(grid_size):
            for j in range(grid_size):
                if board[i][j] == 0:
                    return (i, j)
        return None

    # Backtracking algorithm to solve Sudoku
    def backtrack(board):
        empty_cell = find_empty_cell(board)
        if not empty_cell:
            return True
        else:
            row, col = empty_cell

        for num in range(1, grid_size + 1):
            if is_valid(board, num, (row, col)):
                board[row][col] = num

                if backtrack(board):
                    return True

                board[row][col] = 0

        return False

    return backtrack(board)

This function `solve_sudoku(board, grid_size)` uses a backtracking algorithm to solve a Sudoku puzzle of any specified grid size. Here's a brief description:

- **is_valid(board, num, pos):** Nested function that checks if placing `num` in position `pos` on the Sudoku `board` is valid based on Sudoku rules (no repetition in row, column, or subgrid).

- **find_empty_cell(board):** Nested function that finds an empty cell (represented by `0`) in the Sudoku `board`.

- **backtrack(board):** Recursive function that attempts to solve the Sudoku puzzle:
  - Base case: If no empty cell is found (`find_empty_cell` returns `None`), the puzzle is solved (`True` is returned).
  - Recursive case: Tries placing numbers (`1` to `grid_size`) in the empty cell and recursively attempts to solve the rest of the puzzle.
    - If placing a number leads to a valid solution (`backtrack` returns `True`), the puzzle is solved.
    - If not, the cell is reset (`board[row][col] = 0`) and another number is tried.

- **solve_sudoku(board, grid_size):** Main function that initializes and calls `backtrack(board)` to solve the Sudoku puzzle `board` of size `grid_size x grid_size`. Returns `True` if a solution is found, `False` otherwise.

In [5]:
def is_solution_valid(board, grid_size):
    def is_valid(board, num, pos):
        # Check row and column constraints
        for i in range(grid_size):
            if board[pos[0]][i] == num and pos[1] != i:
                return False
            if board[i][pos[1]] == num and pos[0] != i:
                return False

        # Check subgrid constraints
        subgrid_size = int(grid_size ** 0.5)
        subgrid_x = pos[1] // subgrid_size
        subgrid_y = pos[0] // subgrid_size

        for i in range(subgrid_y * subgrid_size, subgrid_y * subgrid_size + subgrid_size):
            for j in range(subgrid_x * subgrid_size, subgrid_x * subgrid_size + subgrid_size):
                if board[i][j] == num and (i, j) != pos:
                    return False

        return True

    # Check each cell in the board
    for i in range(grid_size):
        for j in range(grid_size):
            if board[i][j] == 0 or not is_valid(board, board[i][j], (i, j)):
                return False

    return True

This function `is_solution_valid(board, grid_size)` checks if a solved Sudoku `board` of a specified `grid_size` is valid according to Sudoku rules. Here's a brief description:

- **is_valid(board, num, pos):** Nested function that checks if placing `num` in position `pos` on the Sudoku `board` is valid based on Sudoku rules:
  - Checks for no repetition of `num` in the same row, column, or subgrid (subgrid size calculated as square root of `grid_size`).

- **is_solution_valid(board, grid_size):**
  - Iterates through each cell in the Sudoku `board`.
  - Checks if each cell is non-zero (`board[i][j] != 0`) and if placing that number in its position is valid using `is_valid`.
  - Returns `True` if all cells are valid according to Sudoku rules; otherwise, returns `False`.

In [6]:
def print_sudoku(board, grid_size):
    for i in range(grid_size):
        if i > 0 and i % int(grid_size ** 0.5) == 0:
            print("-" * (grid_size * 2 + int(grid_size ** 0.5) - 1))

        for j in range(grid_size):
            if j > 0 and j % int(grid_size ** 0.5) == 0:
                print("| ", end="")

            if j == grid_size - 1:
                if board[i][j] == 0:
                    print(" ")
                else:
                    print(board[i][j])
            else:
                if board[i][j] == 0:
                    print(" " + " ", end="")
                else:
                    print(str(board[i][j]) + " ", end="")

This function `print_sudoku(board, grid_size)` prints a Sudoku `board` of a specified `grid_size` in a formatted manner:

- **Functionality:**
  - Iterates through each cell in the Sudoku board.
  - Prints each number in the board, with spaces and separators (`|` and `-`) to visually separate subgrids and rows according to Sudoku grid rules.

- **Details:**
  - Prints horizontal lines (`-`) to separate subgrids after every square root of `grid_size` rows (`int(grid_size ** 0.5)`).
  - Prints vertical separators (`|`) to separate subgrids after every square root of `grid_size` columns (`int(grid_size ** 0.5)`).
  - Prints each number in the board, replacing `0` with a space to indicate empty cells.

In [7]:
def main():
    clear_screen()
    print("Welcome to Sudoku Solver!\n")

    # Get user inputs for grid size and percentage of input
    grid_size = get_user_input("Enter the size of the Sudoku grid (4 or 9): ", [4, 9])
    percentage = get_user_input("Enter the percentage of input you want (30 or 70): ", [30, 70])

    # Load puzzle based on user input
    puzzle = load_puzzle_data(grid_size, percentage)

    # Initialize the Sudoku grid
    sudoku = []
    row = []

    # Convert puzzle string to a 2D list
    for i in range(len(puzzle)):
        row.append(int(puzzle[i]))
        if (i + 1) % grid_size == 0:
            sudoku.append(row)
            row = []

    # Display the initial problem
    clear_screen()
    print("\nProblem: \n")
    print_sudoku(sudoku, grid_size)

    # Wait for user input before displaying the solution
    input("\nPress Enter to See the Solution...")

    # Solve the Sudoku puzzle
    if solve_sudoku(sudoku, grid_size):
        clear_screen()
        print("\nSolution: \n")
        print_sudoku(sudoku, grid_size)

        # Verify the solution
        if is_solution_valid(sudoku, grid_size):
            print("\nThe solution is verified.")
        else:
            print("\nYour solution is incorrect. Please try again.")
    else:
        print("\nNo solution found for this Sudoku puzzle.")

This Python `main()` function provides a complete workflow for solving and verifying a Sudoku puzzle:

- **Functionality:**
  - Clears the screen and greets the user.
  - Prompts the user to enter the grid size (4 or 9) and the percentage of initially filled cells (30 or 70).
  - Loads a Sudoku puzzle based on user input and initializes the Sudoku grid.
  - Displays the initial puzzle using `print_sudoku()`.
  - Waits for the user to press Enter before solving the puzzle.
  - Attempts to solve the Sudoku puzzle using `solve_sudoku()` and displays the solution if found.
  - Verifies the solution's validity using `is_solution_valid()` and provides appropriate feedback.
  - Handles cases where no solution is found for the puzzle.

In [8]:
if __name__ == "__main__":
    main()

Welcome to Sudoku Solver!

Enter the size of the Sudoku grid (4 or 9): 9
Enter the percentage of input you want (30 or 70): 70

Problem: 

  4 5 | 8 1 3 |   2  
9 7 1 | 2 6 5 | 3   8
  2 8 | 9 7   |   6 5
--------------------
1   4 |   2 8 | 6 3  
    3 | 1   9 | 5 8 2
5   2 |   3   | 9    
--------------------
4   9 | 7   6 | 2   3
  5 6 |     2 | 4 7 1
2 3 7 | 4 5 1 | 8 9  

Press Enter to See the Solution...

Solution: 

6 4 5 | 8 1 3 | 7 2 9
9 7 1 | 2 6 5 | 3 4 8
3 2 8 | 9 7 4 | 1 6 5
--------------------
1 9 4 | 5 2 8 | 6 3 7
7 6 3 | 1 4 9 | 5 8 2
5 8 2 | 6 3 7 | 9 1 4
--------------------
4 1 9 | 7 8 6 | 2 5 3
8 5 6 | 3 9 2 | 4 7 1
2 3 7 | 4 5 1 | 8 9 6

The solution is verified.
